In [8]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import urllib
import json
import os
import re
import time
import iNeedCos
import iNeedNet
import jieba
# from pyltp import Segmentor

In [9]:
import socket
socket.setdefaulttimeout(10)

## 1.读入文本

In [10]:
# 读入专家文本
list_car = []
list_txt = []
file_car_list = os.listdir('E:/car2')
for file_car in file_car_list:
    file_txt_list = os.listdir('E:/car2/' + file_car)
    for file_txt in file_txt_list:
        # 写入车名
        list_car.append(file_car)
        # 写入内容
        path = 'E:/car2/' + file_car + '/' + file_txt
        f = open(path,errors='ignore')
        txt = f.read().strip()
        f.close()
        list_txt.append(txt)
data_txt = pd.DataFrame({'company':list_car,'txt':list_txt})

In [11]:
data_txt

,company,txt
0,上汽乘用车,过去，中国车企在海外市场往往走的是价格战的老路。不过，有一个品牌一开始就剑指中高端市场，在海...
1,上汽乘用车,2015年，新能源汽车市场可谓喜忧参半，喜的是实现了爆发性增长前途一片光明，忧的是发展过程中...
2,上汽乘用车,上汽通用汽车作为行业领先企业，以创新求变的发展思路，提出从“卓越经营”到“卓越‘精’营”的战...
3,上汽乘用车,曾经有专家提出，落后国际几十年的中国汽车可以在新能源汽车领域实现“弯道超车”。因为从商业化时...
4,上汽乘用车,本报记者 何芳 实习生 马英歌 北京报道3月13日，在大众集团柏林宣布2017财年业绩的同时...
5,众泰新能源,对于传统能源汽车而言，零部件使用的主要原材料为钢材、石油、有色金属、天胶、布料及其他材料，近...
6,众泰新能源,今年以来，中央与各地政府密集出台了针对新能源汽车的补贴政策，并于9月1日起正式免征新能源汽车...
7,众泰新能源,十、本次交易对中小投资者权益保护的安排\n\n（一）严格履行相关程序并及时履行信息披露义务\...
8,北汽新能源,对于布局“一盘大棋”的北京新能源汽车股份有限公司（以下简称“北汽新能源”）来说，借道“S?鄢...
9,北汽新能源,北京商报讯（记者）1月9日，北汽新能源交出了一份去年的成绩单，产销量双双破10万辆，成为首家...


In [ ]:
# 读入专利文本
list_excel = ['北汽新能源','比亚迪','吉利控股','江淮汽车','江铃新能源',\
             '奇瑞新能源','上海汽车集团','长安新能源','知豆','众泰新能源','威马汽车','蔚来汽车','小鹏汽车']
data_patent = pd.DataFrame()
for excel in list_excel:
    path = 'E://car//专利数据//' + excel + '.xlsx'
    temp_data = pd.read_excel(path,sheet_name=('数据'))
    temp_data = temp_data[[('标题'),('主权项'),('专利类型'),\
                          ('主分类号'),('分类号')]]
    temp_data['company'] = excel
    data_patent = pd.concat([data_patent, temp_data])
data_patent.head()

In [ ]:
data_patent[data_patent['company']=='小鹏汽车']

In [ ]:
# 读入供筛选的专利号,筛选新能源车专利
path = 'E://car//专利数据//专利分类号汇总.xlsx'
data_category = pd.read_excel(path, sheet_name=('分类号汇总'))
list_category = data_category[('汇总')].values
import types
def containCategory(x):
    x = str(x)
    if x.isdigit(): return False ##去掉数字的情况
    this_categories = [c[:4] if len(c)>=4 else c for c in x.split(',')]
    for c in this_categories:
        if c in list_category: return True
    return False
data_patent = data_patent[data_patent[('专利类型')] != ('外观设计')]
data_patent = data_patent[data_patent[('主分类号')].apply(containCategory)]

In [ ]:
# 读入新闻文本
path = 'E:\car\汽车企业数据汇总1124.xlsx'
data_news = pd.read_excel(path, sheet_name=('电动车新闻'))
data_news = data_news[[('标题'),('正文')]]
data_news.head()

In [ ]:
# 读入车辆评论文本
path = 'E:\work\竞争对手\数据\车企\汽车企业数据汇总1124.xlsx'
data_car_comment = pd.read_excel(path.decode('utf8'), sheetname=('口碑').decode('utf8'))
data_car_comment = data_car_comment[[('车型').decode('utf8'),('点评内容').decode('utf8')]]
data_car_comment.head()

In [ ]:
# 读入招聘文本
list_excel = ['猎聘网北汽新能源招聘数据','猎聘网比亚迪招聘数据','猎聘网吉利汽车招聘数据','猎聘网江铃汽车招聘数据','猎聘网奇瑞汽车招聘数据',\
             '猎聘网上汽乘用车招聘数据','猎聘网长安汽车招聘数据','猎聘网知豆招聘数据','猎聘网众泰汽车招聘数据',\
             '猎聘网蔚来汽车招聘数据','猎聘网小鹏汽车招聘数据','猎聘网威马汽车招聘数据']
data_recruit = pd.DataFrame()
for excel in list_excel:
    path = 'E://car//NEV招聘数据//' + excel + '.xlsx'
    temp_data = pd.read_excel(path,sheet_name=('数据'))
    temp_data = temp_data[[('职位名称'),('职位描述'),('所属部门')]]
    temp_data['company'] = excel.replace('猎聘网','').replace('招聘数据','')
    data_recruit = pd.concat([data_recruit, temp_data])
data_recruit.head()

## 2.分词与预处理

### 2.1 分词

In [12]:
txt = '全面负责新能源系列车型的项目质量管理工作① 牵头组织成立项目质量的工作小组；'

In [13]:
# from pyltp import Segmentor
# cws_model_path = 'E:\ltp_data\cws.model'
# segmentor = Segmentor()  # 初始化实例
# segmentor.load(cws_model_path)  # 加载模型
# segmentor.release()  # 释放模型

# 分词
def segment(txt):
    return list(jieba.cut(txt))
#     return list(segmentor.segment(txt))

print(','.join(segment(txt)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\acer\AppData\Local\Temp\jieba.cache
Loading model cost 1.084 seconds.
Prefix dict has been built succesfully.


全面,负责,新能源,系列,车型,的,项目,质量,管理工作,①, ,牵头,组织,成立,项目,质量,的,工作,小组,；


### 2.2 去停用词

In [14]:
## 读入停用词
import re
list_stopwords = []
stop_word_path = 'E:\ltp_data\stopWords.txt'
f = open(stop_word_path, encoding='utf8')
for line in f.readlines():
    list_stopwords.append(line.strip())
f.close()
## 删除停用词
def filter_stopwords(list_segmentwords):
    result = []
    for word in list_segmentwords:
        if word in list_stopwords: continue
        if re.search(r'[0-9]', word) != None: continue
        result.append(word)
    return result

# print(','.join(filter_stopwords(segment(txt))))

### 2.3 同义词归并（暂无）

## 3.生成每个品牌的词袋

In [15]:
# 得到dict_车型_品牌
path = 'E:\car\汽车企业数据汇总1124.xlsx'
data_car_type = pd.read_excel(path, sheet_name=('车型'))
data_car_type = data_car_type[[('品牌'),('型号')]]
dict_cartype_carbrand = dict(zip(data_car_type[('型号')].values, data_car_type[('品牌')].values))

In [16]:
# 得到品牌list
list_brand = list(set(data_car_type[('品牌')].values))

In [17]:
# 需要使用到的函数
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

## 输出领域词表，并返回list_mywords
# def writeMyWords(path, kbest, word):
#     result = []
#     f = open(path, 'w')
#     for i in kbest.get_support(True):
#         result.append(word[i].encode('gb18030'))
#         f.write(word[i].encode('gb18030') + '\r\n')
#     f.close()
#     return result
def writeMyWords(path, kbest, word):
    result = []
#     f = open(path, 'w')
    for i in kbest.get_support(True):
        result.append(word[i])
#         f.write(word[i] + '\r\n')
#     f.close()
    return result

## 只使用筛选后剩下的领域词表
def filterMyWords(list_mywords, list_segmentwords):
    result = []
    for word in list_segmentwords:
        if word in list_mywords: 
            result.append(word)
    return result

## 将各单独文本分词，存入list中
def getListTxtWords(list_txts):
    result = []
    for txt in list_txts:
        this_words = ' '.join(filter_stopwords(segment(txt)))
        result.append(this_words)
    return result

def getListBrandWords(dict_brand_txts):
    result = []
    for brand in dict_brand_txts.keys():
        this_words = ' '.join(filterMyWords(list_mywords, segment(dict_brand_txts[brand])))
        result.append(this_words)
    return result
    

## 将文本tfidf化
### 参数：输入的文本
def getTFIDF(list_txt_words):
    vectorizer = CountVectorizer() #该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频  
    transformer = TfidfTransformer()
    countVec = vectorizer.fit_transform(list_txt_words)
    tfidf = transformer.fit_transform(countVec)
    word = vectorizer.get_feature_names()
    weight_count = countVec.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
    weight_tfidf = tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重
    print('初始特征数：' + str(len(word)))
    return word, tfidf, weight_count, weight_tfidf

## 特征筛选
def selectKBest(tfidf, listTarget, k):
    kbest  = SelectKBest(chi2, k=1000).fit(tfidf, list_target)
    X_new = kbest.transform(tfidf)
    print(X_new.shape)
    return kbest

## 对线的权重最大最小标准化
from sklearn import preprocessing
def minMax(dict_line_weight):
    scale = preprocessing.MinMaxScaler()
    transform = scale.fit_transform(np.fromiter(dict_line_weight.values(),dtype=float).reshape(-1,1))
    for i in range(len(dict_line_weight)):
        dict_line_weight[list(dict_line_weight.keys())[i]] = transform[i][0]
    return dict_line_weight

### 3.0 专家

In [18]:
# 每个品牌的车辆评论的文本
dict_txt_brand_txts = {}
# 逐个文本及其所属类别
list_txts = []
list_target = []
for i in range(len(data_txt)):
    brand = data_txt['company'].values[i]
    this_txt = str(data_txt['txt'].values[i])
    ## 每个品牌的车辆评论文本
    if brand not in dict_txt_brand_txts:
        dict_txt_brand_txts[brand] = ''
    dict_txt_brand_txts[brand] = dict_txt_brand_txts[brand] + this_txt
    ## 逐个文本及其所属类别
    list_txts.append(this_txt)
    list_target.append(brand)

In [19]:
# 将各文本list化，tfidf抽取特征，然后进行特征选择
list_txt_txt_words = getListTxtWords(list_txts)
# 将单独文本tf-idf化
word_txt, tfidf_txt, weight_count_txt, weight_tfidf_txt = getTFIDF(list_txt_txt_words)
# 将特征维度缩减到1000
kbest = selectKBest(tfidf_txt, list_target, 1000)
# 输出词表
path = 'E:\ltp_data\word_txt.txt'
list_mywords = writeMyWords(path, kbest, word_txt)

初始特征数：8317
(58, 1000)


In [20]:
# 将各品牌文本list化，便于tfidf运算
list_txt_brand_words = getListBrandWords(dict_txt_brand_txts)
# 求品牌的tfidf
word_brand, tfidf_brand, weight_count_brand, weight_tfidf_brand = getTFIDF(list_txt_brand_words)

初始特征数：946


In [21]:
# 计算线的权重
dict_txt_node_weight = {}
dict_txt_line_weight = {}
for i in range(len(dict_txt_brand_txts)):
    dict_txt_node_weight[list(dict_txt_brand_txts.keys())[i]] = 1
    for j in range(i+1, len(dict_txt_brand_txts)):
        line = list(dict_txt_brand_txts.keys())[i] + ',' + list(dict_txt_brand_txts.keys())[j]
        dict_txt_line_weight[line] = iNeedCos.cos(weight_tfidf_brand[i,:], weight_tfidf_brand[j, :])

In [22]:
# 对线的权重最大最小标准化
dict_txt_line_weight = minMax(dict_txt_line_weight)

In [23]:
# 输出为网络
path_ucinet = 'E://car//网络2//expert.net'
path_gephi_node = 'E://car//网络2//expert_node.csv'
path_gephi_line = 'E://car//网络2//expert_line.csv'
iNeedNet.iNeedNet(dict_txt_node_weight, dict_txt_line_weight, False, False, path_ucinet, path_gephi_node, path_gephi_line)

### 3.1 专利

In [ ]:
# 每个品牌的车辆评论的文本
dict_patent_brand_txts = {}
# 逐个文本及其所属类别
list_txts = []
list_target = []
for i in range(len(data_patent)):
    brand = data_patent['company'].values[i]
    this_txt = str(data_patent[('主权项')].values[i])
#     this_txt = this_txt if isinstance(this_txt, unicode) else ''
    this_txt = this_txt.replace(('授权'),'')
    ## 每个品牌的车辆评论文本
    if brand not in dict_patent_brand_txts:
        dict_patent_brand_txts[brand] = ''
    dict_patent_brand_txts[brand] = dict_patent_brand_txts[brand] + this_txt
    ## 逐个文本及其所属类别
    list_txts.append(this_txt)
    list_target.append(brand)

In [ ]:
len(list_txts)

In [ ]:
# 将各文本list化，tfidf抽取特征，然后进行特征选择
list_patent_txt_words = getListTxtWords(list_txts)
# 将单独文本tf-idf化
word_txt, tfidf_txt, weight_count_txt, weight_tfidf_txt = getTFIDF(list_patent_txt_words)
# 将特征维度缩减到1000
kbest = selectKBest(tfidf_txt, list_target, 1000)
# 输出词表
path = 'E:\ltp_data\word_patent.txt'
list_mywords = writeMyWords(path, kbest, word_txt)

In [ ]:
# 将各品牌文本list化，便于tfidf运算
list_patent_brand_words = getListBrandWords(dict_patent_brand_txts)
# 求品牌的tfidf
word_brand, tfidf_brand, weight_count_brand, weight_tfidf_brand = getTFIDF(list_patent_brand_words)

In [ ]:
# 计算线的权重
dict_patent_node_weight = {}
dict_patent_line_weight = {}
for i in range(len(dict_patent_brand_txts)):
    dict_patent_node_weight[list(dict_patent_brand_txts.keys())[i]] = 1
    for j in range(i+1, len(dict_patent_brand_txts)):
        line = list(dict_patent_brand_txts.keys())[i] + ',' + list(dict_patent_brand_txts.keys())[j]
        dict_patent_line_weight[line] = iNeedCos.cos(weight_tfidf_brand[i,:], weight_tfidf_brand[j, :])

In [ ]:
# 对线的权重最大最小标准化
dict_patent_line_weight = minMax(dict_patent_line_weight)

In [ ]:
import importlib
importlib.reload(iNeedNet)

In [ ]:
# 输出为网络
path_ucinet = 'E://car//网络2//patent.net'
path_gephi_node = 'E://car//网络2//patent_node.csv'
path_gephi_line = 'E://car//网络2//patent_line.csv'
iNeedNet.iNeedNet(dict_patent_node_weight, dict_patent_line_weight, False, False, path_ucinet, path_gephi_node, path_gephi_line)

### 3.2 新闻

In [ ]:
# 根据标题写入新闻属于哪个车企品牌
## 写入dict{车型关键词-品牌}
dict_cartype_carbrand_news = {}
for i in dict_cartype_carbrand:
    ## 车型部分
    car = i.replace(('新能源').decode('utf8'),'').replace(('能源').decode('utf8'),'').replace(('系列').decode('utf8'),'')
    ## 英文部分
    eng = re.findall(r'[\u4e00-\u9fa5]+',car) ##利用中文未编码，将非中文先筛选出来
    eng = eng[0] if len(eng) == 1 else ''
    ## 中文部分
    cn = car.replace(eng,'')
    ## 品牌部分
    brand = dict_cartype_carbrand[i].replace(('新能源').decode('utf8'),'').replace(('集团').decode('utf8'),'')
    ## 写入dict
    if len(car) > 0: dict_cartype_carbrand_news[car] = dict_cartype_carbrand[i]
    if len(eng) > 0: dict_cartype_carbrand_news[eng] = dict_cartype_carbrand[i]
    if len(cn) > 0: dict_cartype_carbrand_news[cn] = dict_cartype_carbrand[i]
    if len(brand) > 0: dict_cartype_carbrand_news[brand] = dict_cartype_carbrand[i]
## 根据标题确定品牌
def matchBrand(x):
    for i in dict_cartype_carbrand_news:
        if i in x: 
            return dict_cartype_carbrand_news[i]
    return ''
data_news['company'] = data_news[('标题').decode('utf8')].apply(matchBrand) 

In [ ]:
# 每个品牌的车辆评论的文本
dict_news_brand_txts = {}
# 逐个文本及其所属类别
list_txts = []
list_target = []
for i in range(len(data_news)):
    brand = data_news['company'].values[i]
    if brand == '': continue
    this_txt = data_news[('正文').decode('utf8')].values[i]
    this_txt = this_txt if isinstance(this_txt, unicode) else ''
    this_txt = this_txt.replace(('&nbsp').decode('utf8'),'')
    this_txt = re.sub('(\[.+\])','',this_txt)
    this_txt = re.sub(('\（.+\）').decode('utf8'),'',this_txt)
    this_txt = re.sub(('查看同类文章.+').decode('utf8'),'',this_txt)
    ## 每个品牌的车辆评论文本
    if dict_news_brand_txts.has_key(brand) == False:
        dict_news_brand_txts[brand] = ''
    dict_news_brand_txts[brand] = dict_news_brand_txts[brand] + this_txt
    ## 逐个文本及其所属类别
    list_txts.append(this_txt)
    list_target.append(brand)

In [ ]:
# 将各文本list化，tfidf抽取特征，然后进行特征选择
list_news_txt_words = getListTxtWords(list_txts)
# 将单独文本tf-idf化
word_txt, tfidf_txt, weight_count_txt, weight_tfidf_txt = getTFIDF(list_news_txt_words)
# 将特征维度缩减到1000
kbest = selectKBest(tfidf_txt, list_target, 1000)
# 输出词表
path = 'E:\ltp_data\word_news.txt'
list_mywords = writeMyWords(path, kbest, word_txt)

In [ ]:
# 将各品牌文本list化，便于tfidf运算
list_news_brand_words = getListBrandWords(dict_news_brand_txts)
# 求品牌的tfidf
word_brand, tfidf_brand, weight_count_brand, weight_tfidf_brand = getTFIDF(list_news_brand_words)

In [ ]:
# 计算线的权重
dict_news_node_weight = {}
dict_news_line_weight = {}
for i in range(len(dict_news_brand_txts)):
    dict_news_node_weight[dict_news_brand_txts.keys()[i]] = 1
    for j in range(i+1, len(dict_news_brand_txts)):
        line = dict_news_brand_txts.keys()[i] + ',' + dict_news_brand_txts.keys()[j]
        dict_news_line_weight[line] = iNeedCos.cos(weight_tfidf_brand[i,:], weight_tfidf_brand[j, :])

In [ ]:
# 对线的权重最大最小标准化
dict_news_line_weight = minMax(dict_news_line_weight)

In [ ]:
# 输出为网络
path_ucinet = 'E://work//竞争对手//数据//车企//网络//news.net'
path_gephi_node = 'E://work//竞争对手//数据//车企//网络//news_node.csv'
path_gephi_line = 'E://work//竞争对手//数据//车企//网络//news_line.csv'
iNeedNet.iNeedNet(dict_news_node_weight, dict_news_line_weight, False, False, path_ucinet, path_gephi_node, path_gephi_line)

### 3.3 车辆评论

In [ ]:
# 每个品牌的车辆评论的文本
dict_comment_brand_txts = {}
# 逐个文本及其所属类别
list_txts = []
list_target = []
for i in range(len(data_car_comment)):
    brand = dict_cartype_carbrand[data_car_comment[('车型').decode('utf8')].values[i].split('-')[1]]
    this_txt = data_car_comment[('点评内容').decode('utf8')].values[i]
    this_txt = this_txt if isinstance(this_txt, unicode) else ''
    this_txt = this_txt.replace(('【最满意的一点】').decode('utf8'),'')\
        .replace(('【最不满意的一点】').decode('utf8'),'')\
        .replace(('【空间】').decode('utf8'),'')\
        .replace(('【动力】').decode('utf8'),'')
    ## 每个品牌的车辆评论文本
    if dict_comment_brand_txts.has_key(brand) == False:
        dict_comment_brand_txts[brand] = ''
    dict_comment_brand_txts[brand] = dict_comment_brand_txts[brand] + this_txt
    ## 逐个文本及其所属类别
    list_txts.append(this_txt)
    list_target.append(brand)

In [ ]:
# 将各文本list化，tfidf抽取特征，然后进行特征选择
list_comment_txt_words = getListTxtWords(list_txts)
# 将单独文本tf-idf化
word_txt, tfidf_txt, weight_count_txt, weight_tfidf_txt = getTFIDF(list_comment_txt_words)
# 将特征维度缩减到1000
kbest = selectKBest(tfidf_txt, list_target, 1000)
# 输出词表
path = 'E:\ltp_data\word_comment.txt'
list_mywords = writeMyWords(path, kbest, word_txt)

In [ ]:
# 将各品牌文本list化，便于tfidf运算
list_comment_brand_words = getListBrandWords(dict_comment_brand_txts)
# 求品牌的tfidf
word_brand, tfidf_brand, weight_count_brand, weight_tfidf_brand = getTFIDF(list_comment_brand_words)

In [ ]:
# 计算线的权重
dict_comment_node_weight = {}
dict_comment_line_weight = {}
for i in range(len(dict_comment_brand_txts)):
    dict_comment_node_weight[dict_comment_brand_txts.keys()[i]] = 1
    for j in range(i+1, len(dict_comment_brand_txts)):
        line = dict_comment_brand_txts.keys()[i] + ',' + dict_comment_brand_txts.keys()[j]
        dict_comment_line_weight[line] = iNeedCos.cos(weight_tfidf_brand[i,:], weight_tfidf_brand[j, :])

In [ ]:
# 对线的权重最大最小标准化
dict_comment_line_weight = minMax(dict_comment_line_weight)

In [ ]:
# 输出为网络
path_ucinet = 'E:\work\竞争对手\数据\车企\网络\comment.net'
path_gephi_node = 'E:\work\竞争对手\数据\车企\网络\comment_node.csv'
path_gephi_line = 'E:\work\竞争对手\数据\车企\网络\comment_line.csv'
iNeedNet.iNeedNet(dict_comment_node_weight, dict_comment_line_weight, False, False, path_ucinet, path_gephi_node, path_gephi_line)

### 3.4 招聘

In [ ]:
# 每个品牌的车辆评论的文本
dict_recruit_brand_txts = {}
# 逐个文本及其所属类别
list_txts = []
list_target = []
for i in range(len(data_recruit)):
    brand = data_recruit['company'].values[i]
    this_txt = data_recruit[('职位描述')].values[i]
#     this_txt = this_txt if isinstance(this_txt, unicode) else ''
    this_txt = this_txt.replace(('职位描述'),'')\
        .replace(('岗位职责'),'')\
        .replace(('工作职责'),'')
    ## 每个品牌的车辆评论文本
    if brand not in dict_recruit_brand_txts:
        dict_recruit_brand_txts[brand] = ''
    dict_recruit_brand_txts[brand] = dict_recruit_brand_txts[brand] + this_txt
    ## 逐个文本及其所属类别
    list_txts.append(this_txt)
    list_target.append(brand)

In [ ]:
# 将各文本list化，tfidf抽取特征，然后进行特征选择
list_recruit_txt_words = getListTxtWords(list_txts)
# 将单独文本tf-idf化
word_txt, tfidf_txt, weight_count_txt, weight_tfidf_txt = getTFIDF(list_recruit_txt_words)
# 将特征维度缩减到1000
kbest = selectKBest(tfidf_txt, list_target, 1000)
# 输出词表
path = 'E:\ltp_data\word_recruit.txt'
list_mywords = writeMyWords(path, kbest, word_txt)

In [ ]:
# 将各品牌文本list化，便于tfidf运算
list_recruit_brand_words = getListBrandWords(dict_recruit_brand_txts)
# 求品牌的tfidf
word_brand, tfidf_brand, weight_count_brand, weight_tfidf_brand = getTFIDF(list_recruit_brand_words)

In [ ]:
# 计算线的权重
dict_recruit_node_weight = {}
dict_recruit_line_weight = {}
for i in range(len(dict_recruit_brand_txts)):
    dict_recruit_node_weight[list(dict_recruit_brand_txts.keys())[i]] = 1
    for j in range(i+1, len(dict_recruit_brand_txts)):
        line = list(dict_recruit_brand_txts.keys())[i] + ',' + list(dict_recruit_brand_txts.keys())[j]
        dict_recruit_line_weight[line] = iNeedCos.cos(weight_tfidf_brand[i,:], weight_tfidf_brand[j, :])

In [ ]:
# 对线的权重最大最小标准化
dict_recruit_line_weight = minMax(dict_recruit_line_weight)

In [ ]:
# 输出为网络
path_ucinet = 'E://car//网络2//recruit.net'
path_gephi_node = 'E://car//网络2//recruit_node.csv'
path_gephi_line = 'E://car//网络2//recruit_line.csv'
iNeedNet.iNeedNet(dict_recruit_node_weight, dict_recruit_line_weight, False, False, path_ucinet, path_gephi_node, path_gephi_line)

# N.N inBook论文代码（借用下）

In [ ]:
# 文字识别关键代码
access_token = read_access_token() #获取token
url = 'https://aip.baidubce.com/rest/2.0/ocr/v1/general_basic?access_token=' + access_token #构建接口url
img=base64.b64encode(f.read()) #读取待识别照片
response = urllib2.urlopen(request) #发送请求
content = response.read() #获得初步识别结果

In [ ]:
# 命名实体识别关键代码
result = urllib.urlopen(url_get_base, urllib.urlencode(args)) #构建请求参数，从LTP语言云获取命名实体识别结果
content = result.read() #读取内容
list_word = [item.split(' ')[0].decode('utf-8') for item in content.split('\n')] #识别出的命名实体列表
jieba.load_userdict('..\\data\\entity.txt') #加载读者词典
for w in pseg.cut(txt): #使用jieba分词第二次抽取命名实体
    if (w.flag == 'ns') or (w.flag == 'nh') or (w.flag == 'ni') or (w.flag == 'nz'):
        f len(w.word) < 2: continue #实体最短长度为2
result.append(w.word)

In [ ]:
# 聚合检索关键代码
r=requests.get("http://data1.library.sh.cn/sg/work/data?keyword="+word+"&key=***") #从上图开放接口检索数据
soup = requestSoup("https://book.douban.com/subject_search?search_text=" + word + "&cat=1001") #构建soup，访问互联网资源
for result in soup.find_all(class_='subject-item', limit=count):#抽取标题
#...# #抽取内容、图片等内容
this_search_list.append(search)

In [ ]:
# 知识网络生成关键代码
$output_node = implode("", $node); #导入已经查询好的点和线数据
$output_link = implode("", $lnks);
# 使用echarts绘制网络图 
$outputjs = 'var page = require("webpage").create();
page.open("https://www.zhuomutech.com/net/'.$userID.'_net.html", function(status) {
console.log("Status: " + status);
if(status === "success") {
page.render("net/'.$userID.'_'.$timename.'_net.png");
}
phantom.exit();
});';
file_put_contents("net/run_".$userID."_net.js", $outputjs);
$torf = exec('phantomjs E:/thinkphp_5.0.10_full/public/net/run_'.$userID.'_net.js'); #使用phantom模拟打开浏览器，保存网络图到服务器指定路径